<a href="https://colab.research.google.com/github/DiegoBores/TutorialBERT/blob/main/TutorialBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=gTM3hOHW4hUY

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 26.0 MB/s 
     |████████████████████████████████| 6.5 MB 51.4 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#importamos las librerias necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importamos el dataset
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [ ]:
#solo utilizaremos las 2000 primeras reviews
batch_1 = df[:2000]

In [ ]:
##importante para el analisis de los datos:
#pedimos a pandas saber cuantas review son positivas (un 1) y cuantas son negativas (un 0)
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [ ]:
#Cargamos el modelos pre-entrenado de distilBERT
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#Para utilizar BERT utilizariamos la siguiente sentencia
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

#Cargamos el mdelo/tokenizer preentrenado
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Para el modelo 1, que realiza un procesado de las frases que despues se pasa a un modelo de regresion logistica:
#Primero preparamos el DATASET

#Tokenization: dividir las frases de review en palabras y subpalabras en el formato que exige BERT
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
#Padeo: queremos que BERT procese nuestros ejemplos a la vez, como un solo batch. Para ello, debemos padear todas las listas al mismo tamaño, para poder representarlo como un array 2D

max_len = 0
#Obtenemos el valor de la mayor longitud de las listas
for i in tokenized.values:
  if len(i)> max_len:
    max_len = len(i);
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

#Ahora nuestro dataset se encuentra dentro de padded, y sus dimensiones son:
np.array(padded).shape 

(2000, 59)

In [ ]:
#masking: debemos indicas a BERT que debe ignorar el padeo introducido anteriormente. Esta informacion se almacenara en la variable attention_mask:
attention_mask = np.where(padded!=0 , 1, 0)
attention_mask.shape

(2000, 59)

In [ ]:
#Por ultimo para el modelo 1 vamos a correrlo. la funcion model()corre nuestras sentencias.

inputs_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(inputs_ids, attention_mask = attention_mask)

In [ ]:
#Ahora seccionamos solo la parte de la salida que necesitamos. ESto es, la salida correpondiente al primer token de cada review. 
#La forma en que BERT hace la clasificacion de reviews es añadiendo un token llamado [CLS] al principio de cada frase. 
#La salida correspondiente a ese token se puede entender como un embebido de la frase completa

#Ahora guardamos esto en la variable features, ya que esto servirá como las variables para nuestro modelo de regresion logística:
features = last_hidden_states[0][:,0,:].numpy()

#Las etiquetas indicando que frase es positiva y cual negativa va en la variable labels

labels = batch_1[1]

In [ ]:
#Modelo 2: recoge el resultado de BERT y clasifica las frases en positiva o negativa

#Primero deividimos el dataset e conjunto de entrenamiento y de test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
#Realizamos una búsqueda de los mejores parametros para la regresion logistica de Scikit Learn
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scores:  0.8226666666666667


In [ ]:
#Ahora entrenamos el modelos de Regresion Logistica. Utilizamos el valor de C obtenido
lr_clf=LogisticRegression(C=5.263252)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.263252)

In [ ]:
#Por ultimo, evaluamos el modelo
lr_clf.score(test_features, test_labels)

0.85

In [ ]:
#Lo compararemos con un clasificador sencillo:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()
scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std()*2))

Dummy classifier score: 0.533 (+/- 0.00)
